In [2]:
import torch
from huggingface_hub import Repository

In [3]:
path_to_home = "../"
#path_to_home = "./drive/MyDrive/"

In [3]:
hf_repo = Repository(path_to_home + "hugging_face_repo", clone_from="https://huggingface.co/sibrun/receiptlayoutlm")

/Users/simon/PycharmProjects/receiptlayoutlm/code/../hugging_face_repo is already a clone of https://huggingface.co/sibrun/receiptlayoutlm. Make sure you pull the latest changes with `repo.git_pull()`.


In [4]:
hf_repo = Repository(path_to_home + "hugging_face_repo")
#hf_repo.git_pull()

In [5]:
from datasets import load_from_disk, load_dataset

ds_receipts = load_from_disk(path_to_home + "datasets/ds_receipts_final")
#ds_receipts = load_dataset("sibrun/receipts", use_auth_token=True)
ds_receipts['train'].features

{'image': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='uint8', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'bbox': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [6]:
from datasets import Features, Array2D, Array3D, Sequence, Value
max_length = 512
features = Features({
    'image': Array3D(dtype="uint8", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype="int32"), length=max_length),
    'bbox': Array2D(dtype="int64", shape=(max_length, 4)),
    'labels': Sequence(feature=Value(dtype="int64"), length=max_length),
    'attention_mask': Sequence(feature=Value(dtype="int8"), length=max_length),
})

In [7]:
ds_receipts = ds_receipts.cast(features)
ds_receipts['train'].features

Loading cached processed dataset at ../datasets/ds_receipts_final/train/cache-e5c271fb43f54ac8.arrow
Loading cached processed dataset at ../datasets/ds_receipts_final/test/cache-c1bc42bb40029ecf.arrow


{'image': Array3D(shape=(3, 224, 224), dtype='uint8', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=512, id=None),
 'bbox': Array2D(shape=(512, 4), dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=512, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=512, id=None)}

In [8]:
label_names = ['company', 'date', 'address', 'total']
labels = ['O'] + label_names
num_labels = len(labels)
ids_to_labels = {k: v for k, v in enumerate(labels)}
labels_to_ids = {v: k for k, v in enumerate(labels)}

In [9]:
from transformers import AutoConfig
from transformers import LayoutLMv2ForTokenClassification

xlm_config = AutoConfig.from_pretrained("microsoft/layoutxlm-base",
                                         num_labels=num_labels,
                                         id2label=ids_to_labels,
                                         label2id=labels_to_ids)

In [10]:
model_xlm = LayoutLMv2ForTokenClassification.from_pretrained("microsoft/layoutxlm-base", config=xlm_config)

Some weights of the model checkpoint at microsoft/layoutxlm-base were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.0.shortcut.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.15.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.10.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.0.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.22.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.8.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.19.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.3.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.18.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.1.conv1.norm.num_batches_tracked', 'l

In [11]:
model_xlm.save_pretrained(path_to_home + "models/token_class_xlm_base")

In [12]:
model_xlm = LayoutLMv2ForTokenClassification.from_pretrained(path_to_home + "models/token_class_xlm_base")

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ds_receipts.set_format(type="torch", device=device)
model_xlm.to(device)

LayoutLMv2ForTokenClassification(
  (layoutlmv2): LayoutLMv2Model(
    (embeddings): LayoutLMv2Embeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visual): LayoutLMv2VisualBackbone(
      (backbone): FPN(
        (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 

In [21]:
from argparse import Namespace

config = {"train_batch_size": 4,
          "valid_batch_size": 2,
          "weight_decay": 0.1,
          "learning_rate": 5e-5,
          "num_train_epochs": 2,
          "seed": 1,
          "save_checkpoint_steps": 100}

args = Namespace(**config)

logging_file_path = path_to_home + "log/logging/train.log"
wandb_dir = path_to_home +"log/wandb"
tensorboard_dir = path_to_home + "log/tensorboard"
hf_models_dir = path_to_home + "models/receiptlayoutlm"
project_name = "receiptlayoutlm"
user_name = "sibrun"
hf_model = user_name + "/" + project_name

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(ds_receipts['train'], batch_size=args.train_batch_size, shuffle=True)
eval_dataloader = DataLoader(ds_receipts['test'], batch_size=args.valid_batch_size)

In [23]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k, v.shape)

image torch.Size([4, 3, 224, 224])
input_ids torch.Size([4, 512])
bbox torch.Size([4, 512, 4])
labels torch.Size([4, 512])
attention_mask torch.Size([4, 512])


In [24]:
optimizer = torch.optim.AdamW(model_xlm.parameters(), lr=args.learning_rate)

In [25]:
from tensorboardX import SummaryWriter
import logging
#import wandb
#import transformers
#import datasets

def setup_logging():
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
        handlers=[logging.FileHandler(logging_file_path), logging.StreamHandler()])
    #wandb.init(project=project_name_wandb,
    #           config=args,
    #           dir=wandb_dir,
    #           settings=wandb.Settings(start_method="fork"),
    #           entity="sibrun",
    #           sync_tensorboard=True)
    #run_name = wandb.run.name
    tb_writer = SummaryWriter()
    tb_writer.add_hparams(vars(args), {'0': 0})
    logger.setLevel(logging.INFO)
    #datasets.utils.logging.set_verbosity_debug()
    #transformers.utils.logging.set_verbosity_info()
    return logger, tb_writer #, run_name

In [26]:
def log_metrics(step, metrics, tb_writer):
    logger.info(f"Step {step}: {metrics}")
    #wandb.log(metrics)
    [tb_writer.add_scalar(k, v, step) for k, v in metrics.items()]

In [27]:
def evaluate():
    model_xlm.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            model_outputs = model_xlm(**batch)
        losses.append(model_outputs.loss)
    total_loss = torch.mean(torch.stack(losses))
    try:
        perplexity = torch.exp(total_loss)
    except OverflowError:
            perplexity = torch.tensor(float("inf"))
    return loss.item(), perplexity.item()

In [28]:
from random import seed
seed(args.seed)

logger, tb_writer = setup_logging()

In [29]:
step = 0
model_xlm.train()
for epoch in range(args.num_train_epochs):
   print("Epoch:", epoch)
   for batch in train_dataloader:
       optimizer.zero_grad()
       outputs = model_xlm(**batch)
       loss = outputs.loss
       log_metrics(step, {'steps': step, 'loss/train': loss.item()}, tb_writer)
       if step % 10 == 0:
           print(f"Loss after {step} steps: {loss.item()}")
       if step % args.save_checkpoint_steps == 0:
           logger.info('Evaluating and saving model checkpoint')
           eval_loss, perplexity = evaluate()
           log_metrics(step, {'loss/eval': eval_loss, 'perplexity': perplexity}, tb_writer)
           model_xlm.save_pretrained(hf_models_dir)
           model_xlm.push_to_hub(hf_model, commit_message=f'step {step}')
       model_xlm.train()
       loss.backward()
       optimizer.step()
       step += 1

logger.info('Evaluating and saving model after training')
eval_loss, perplexity = evaluate()
log_metrics(step, {'loss/eval': eval_loss, 'perplexity': perplexity}, tb_writer)
model_xlm.save_pretrained(hf_models_dir)
model_xlm.push_to_hub(hf_model, commit_message='final model')

Epoch: 0


/Users/simon/PycharmProjects/receiptlayoutlm/venv/lib/python3.9/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:772: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  torch.arange(
/Users/simon/PycharmProjects/receiptlayoutlm/venv/lib/python3.9/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:782: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, us

Loss after 0 steps: 1.6082464456558228


03/13/2022 20:17:35 - INFO - __main__ - Step 0: {'loss/eval': 1.6082464456558228, 'perplexity': 4.877651691436768}


Upload file pytorch_model.bin:   0%|          | 32.0k/1.38G [00:00<?, ?B/s]

KeyboardInterrupt: 